# First and second order HMM for typos correction 
## Belouadah Eden & Bouhaha Mariem 

### Introduction

Le but de ce projet est de mettre en pratique le modèle HMM avec l'algorithme de Viterbi pour la correction des fautes de frappes. 
Les observations dans la chaine de Markov sont les lettres tapées par l'utilisateur et les états (cachés) sont les vraies lettres correspondantes.  Le but étant de corriger la séquence des lettres, c'est à dire d'essayer de trouver la séquence des états cachés.

La base de données utilisée pour ce projet est générée à partir du document "Nabomber's Manifesto". Deux variantes sont mises à notre disposition. Etant donné le text d'origine, la première variante ("train10" et "test10") est obtenue en donnant une probabilité de 0.1 pour que la lettre soit aléatoirement générée à la place de la lettre originale; et la deuxième variante ("train20" et "test20") est obtenue en donnant une probabilité de 0.2 pour que la lettre soit aléatoirement générée à la place de la lettre originale. La deuxième variante n'est qu'une version plus difficile de la première.

### HMM du premier ordre

Le HMM du premier ordre s'appuie sur l'hypothèse que l'état courant ne dépend que de l'état précédent. Cette hypothèse simplificatrice permet d'avoir un modèle simple, rapide et efficace.

On commence par importer les modules nécessaires:

In [35]:
import numpy as np #Pour la gestion des tableaux et des matrices
import pickle #Pour la lecture de la base de données
from nltk.probability import FreqDist #Pour les dictionnaires de comptes

Chargement de la base de données: Pour changer la base de données, il suffit de choisir ("train10" et "test10") ou bien ("train20" et "test20").

In [36]:
ensApprentissage = pickle.load(open('train10.pkl', 'rb'))
ensTest = pickle.load(open('test10.pkl', 'rb'))

Initialisation des hyper-paramètres du modèle:

In [37]:
inconnu = 0  #Encodage des mots inconnus (peu fréquents)
facteurLissage = 0.0001 #Pour éviter d'avoir un dénominateur nul. 

Comptage du nombre d'observations (lettres tapées) possibles, le nombre d'états (lettres corrects) possibles, le nombre de transitions possibles entre les états,  le nombre de fois où chaque état apparait au début d'une séquence (mot) et le nombre de pairs d'états possibles:

In [38]:
cptTapees = FreqDist() #Nombre d'observations possibles
cptCorrectes = FreqDist() #Nombre d'états possibles
cptBigramCorrectes = FreqDist() #Nombre de pairs d'états
cptTransitions = FreqDist() #Nombre de transitions entre états
cptDebut = FreqDist() #Nombre de fois où les états apparaissent au début d'un mot
for mot in ensApprentissage: #Pour chaque mot de l'ensemble d'apprentissage
    for i in range(len(mot)): #Pour chaque lettre du mot (la lettre contient l'observation+état caché)
        tapee_correcte = mot[i] #Récupérer le couple (observation,état)
        tapee = tapee_correcte[0] #Observation
        correcte = tapee_correcte[1] #Etat
        #Compteur des lettres tapées (les observations)
        cptTapees[tapee] += 1
        #Compteur des lettres correctes (les états)
        cptCorrectes[correcte] += 1
        #Compteur des bigrammes des états
        cptBigramCorrectes[tapee_correcte] += 1
        if i==0: #Compteur des états initiaux
            cptDebut[correcte] += 1       
        else: #Compteur des transitions entre les états
            cptTransitions[(mot[i-1][1], correcte)] += 1

Récupérer la liste des observations possibles, la liste des états possibles et leurs tailles

In [39]:
observations=list(cptTapees.keys())#Les observations sont les clés du dictionnaire des comptes  des lettres tapées
etats=list(cptCorrectes.keys())#Les états sont les clés du dictionnaire des compte des lettres correctes
N=len(etats)#Nombre d'états
M=len(observations)#Nombre d'observations

Créer des dictionnaires qui permettent de passer d'une lettre (observation/état) à son encodage en entier correspondant

In [40]:
dictTapees = {}
dictCorrectes = {}
#Coder l'état par sa position
for i in range(len(etats)):
    dictCorrectes[etats[i]] = i
for i in range( len(observations)):
    dictTapees[observations[i]] = i

Calculer A (matrice des probabilités de transitions), B (matrice de probabilités d'observations) et Pi (vecteur des probabilités initiales)

In [41]:
A = np.zeros((N,N), float) #matrice des probabilités de transitions
B = np.zeros((M,N), float) #matrice de probabilités d'observations
Pi = np.zeros((N,), float) #Vecteur des probabilités initiales des états
#######################################################
#Vecteur Pi
for correcte in cptDebut: #Pour chaque état
    i = dictCorrectes[correcte] #Récupérer sa position
    Pi[i] = cptDebut[correcte] #Récupérer son nombre d'apparition
Pi = Pi / sum(Pi) #Calculer la moyenne
########################################################
#Matrice A:
for tapee_correcte in cptTransitions: 
    i = dictCorrectes[tapee_correcte[1]] #Récupérer la position de l'état
    j = dictCorrectes[tapee_correcte[0]] #Récupérer la position de l'observation
    A[i, j] = cptTransitions[tapee_correcte] #Récupérer leur nombre d'apparition ensemble
A = A / A.sum(axis=0).reshape(1, N) #Calculer la moyenne
########################################################
#Matrice B:
for tapee_correcte in cptBigramCorrectes:
    tapee = tapee_correcte[0] #Récupérer l'observation
    correcte = tapee_correcte[1] #Récuoérer l'état
    if tapee in dictTapees: #On fait cette vérification parce qu'il peut  y avoir des caractères non alphabétics introduits par erreur de frappe
        i = dictTapees[tapee] #Récupérer la position de l'observation
    j = dictCorrectes[correcte] #Pas de vérification d'appartenance car on est sûr que c'est vérifié
    B[i, j] = cptBigramCorrectes[tapee_correcte] #Récupérer le nombre d'apparition
B = B + facteurLissage #Ajouter le facteur de lissage
B = B / B.sum(axis=0).reshape(1, N) #Calculer la moyenne

Fonction qui permet de transformer chaque lettre d'un mot en entrée en un entier correspondant unique, pour pouvoir indexer les matrices de probabilités

In [42]:
def CoderMot(mot, dictTapees, dictCorrectes, inconnu):
    codeTapees = []
    codeCorrectes = []
    for tapee_correcte in mot: #Pour chaque couple (état, observation)
        tapee = tapee_correcte[0]
        correcte = tapee_correcte[1]
        if tapee in dictTapees:
            codeTapees.append(dictTapees[tapee]) #Récupérer le code correspondant à l'observation si elle existe dans le dictionnaire
        else:
            codeTapees.append(inconnu) #Mettre le code du caractère inconnu (0)
        codeCorrectes.append(dictCorrectes[correcte]) #Récupérer le code correspondant à l'état
    return codeTapees, codeCorrectes

Essai d'un classifieur stupide qui ne fait rien, donc il suffit de calculer le taux des lettres erronées parmis les lettres de la séquence observée.

In [43]:
cptBienCorrigees = 0
somme = 0
for mot in ensTest: #Pour chaque mot de l'ensemble de test
    motTape, motCorrect = CoderMot(mot, dictTapees, dictCorrectes, inconnu) #Récupérer le mot en tant que codes
    cptBienCorrigees += sum(np.array(motCorrect)==np.array(motTape)) #Accumuler le nombre de lettres bien corrigés
    somme += len(motTape) #Accumuler le nombre total de lettres 
print ("Précision d'un classifieur stupide = {:.2f}% ".format(float(cptBienCorrigees)/somme*100))

Précision d'un classifieur stupide = 89.82% 


Fonction qui implémente l'algorithme Viterbi, qui permet, à partir d'une séquence de lettres en entrées, de produire la séquences de de lettres cachées correspondantes:

In [44]:
def Viterbi1(motTape, N, A, B, Pi):
    T = len(motTape)
    psi = np.zeros((T, N), int) #Matrice qui va contenir le poids de chaque etat pour chaque lettre du mot tape
    delta_courant = np.zeros(N, float) #On ne garde que deux vecteurs (précédent et courant) pour parcourir psi
    delta_precedent = np.zeros(N, float)
    etats_caches = [] #C'est le resultat qu'on veut construire
    delta_courant = B[motTape[0]] * Pi #Initialisation pour la première itération
    for t in range(1, T): #Pour le reste des itérations
        lettre_courante = motTape[t] #Récupérer la lettre courante du mot tape
        for j in range(N): #Pour chaque état possible
            interm=delta_courant * A[j, :]
            psi[t, j] = interm.argmax() #Récupérer le meilleur état caché
            delta_precedent[j] = interm[interm.argmax()] * B[lettre_courante, j]
        #Permutation:
        z=delta_precedent
        delta_precedent=delta_courant
        delta_courant=z
    #Choisir le meilleur état caché pour la dernière observation
    etats_caches.append(delta_courant.argmax())
    for i in range(psi.shape[0]-1, 0, -1): #Récupérer les meilleurs états des observations précédentes
        etats_caches.append(psi[i, etats_caches[-1]])
    etats_caches.reverse() #Inverser la liste et la renvoyer
    return etats_caches

Essai d'un classifieur HMM du premier ordre:

In [45]:
cptBienCorrigees = 0
somme = 0
for mot in ensTest: #Pour chaque mot de l'ensemble de test
    motTape, motCorrect=CoderMot(mot, dictTapees, dictCorrectes, inconnu) #Récupérer le mot en tant que codes
    motCorrige = Viterbi1(motTape, N, A, B, Pi) #Récupérer la séquence d'états cachés retournés par l'algorithme de Viterbi
    cptBienCorrigees += sum(np.array(motCorrect)==np.array(motCorrige)) #Accumuler le nombre de lettres bien corrigés
    somme += len(motTape) #Accumuler le nombre total de lettres 
print("Précision de l'HMM du premier ordre={:.2f}%".format(float(cptBienCorrigees)/somme*100))
print ("Nombre de lettres bien corrigées = " + str(cptBienCorrigees))
print ("Nombre de lettres mal corrigées = " + str(somme-cptBienCorrigees))

Précision de l'HMM du premier ordre=93.20%
Nombre de lettres bien corrigées = 6822
Nombre de lettres mal corrigées = 498


Nous remarquons que le classifieur Markovien du premier ordre est meilleur que le classifieur stupide. En effet ce dernier a donné une précision de 89.82% sur l'ensemble de test alors que HMM1 a donné 93.20% Mais cela n'est pas vraiment suffisant, car l'écrat entre les deux n'est pas grand. Ceci dit qu'on préfère ne rien faire et obtenir un score de 89% que d'entraîner tout un modèle "couteûx" en temps et en calcul pour avoir un score de 93%. Cela reste bien sur dans le cadre de ce problème seulement et n'est pas une généralisation.

## HMM du deuxième ordre

Dans ce modèle, nous ne tenons pas compte seulement de la lettre précédente comme historique mais plutôt des deux lettres précédentes afin de rendre le modèle un peu plus complexe mais plus efficace


La base de données étant déja chargée et certains comptes sont déjà calculés, nous nous contentons d'ajouter les comptes des trigrammes de lettres et les compte de transitions trigrammes

In [46]:
cptTrigramCorrectes=FreqDist()
cptTransitionsTrigram=FreqDist()
for mot in ensApprentissage: #Pour chaque mot de l'ensemble d'apprentissage
    for i in range(len(mot)):
        tapee_correcte=mot[i]
        tapee = tapee_correcte[0] #Récupérer l'observation
        correcte = tapee_correcte[1] #Récupérer l'état
        #Compteur des trigrammes
        if i > 0 and i<len(mot)-1:
            cptTrigramCorrectes[(mot[i-1][1],correcte)]+=1
        #Compteur des transitions entre les trigrammes
        if i > 1:
            cptTransitionsTrigram[(mot[i-2][1],mot[i-1][1],correcte)]+=1

Récupération la liste des trigrammes possibles des états et sa taille:

In [47]:
trigrammes=list(cptTrigramCorrectes.keys())#Les trigrammes sont les clés du dictionnaire des comptes des trigrammes d'états
L = len(trigrammes) #Longueur de la liste des trigrammes

Créer le dictionnaire d'encodage des trigrammes:

In [48]:
dictTrigram = {}
for i in range(L): #Encoder le trigramme par sa position
    dictTrigram[trigrammes[i]] = i

Les matrices de probabilités restent les mêmes sauf qu'il faut rajouter une matrice pour les transition trigrammes:

In [49]:
C = np.zeros((N, L), float) #Matrice des transitions des trigrammes
for trigramme in cptTransitionsTrigram:
    i=dictCorrectes[trigramme[2]] #Récupérer la position de l'état
    j=dictTrigram[(trigramme[0],trigramme[1])] #Récupérer la position de l'observation1 et l'observation2
    C[i,j]=cptTransitionsTrigram[trigramme] #Calculer le nombre d'appartition
C=C/C.sum(axis=0).reshape(1,L) #Faire la moyenne

Tous les comptes et les probabilités nécessaires sont calculées, l'algorithme de Viterbi devient:

In [50]:
def Viterbi2(motTape, N, L, A, B, C, Pi, dictTrigram, observations, inconnu):
    T = len(motTape)
    delta_courantt = np.zeros( N, float )
    delta_precedent = np.zeros( N, float )
    psi = np.zeros( (T, N), int )      
    #Initialisation pour la première itération
    delta_courant = B[motTape[0]] * Pi 
    etats_caches = [] #Liste des états cachés qu'on veut construire
    for t in range(1, T): #Pour chaque lettre du mot tapé à partir de la deuxième
        if (t==1): #S'il s'agit de la deuxième lettre
            lettre_courante = motTape[t] #Récupérer la lettre
            for j in range(N): #Pour chaque état possible
                interm=delta_courant * A[j,:]
                psi[t, j] = interm.argmax()       
                delta_precedent[j] = interm[interm.argmax()] * B[lettre_courante, j] 
            #Permutation:
            z=delta_courant
            delta_courant=delta_precedent
            delta_precedent=z
        else: #S'il s'agit de la troisième lettre ou plus
            lettre_courante = motTape[t] #Récupérer la lettre
            for j in range(N):
                for x in range(N):
                    y=psi[t-1,x]
                    bigramObservations=(observations[y],observations[x])
                    if bigramObservations in dictTrigram:
                        interm[x]=delta_courant[x]*C[j,dictTrigram[bigramObservations]]
                    else:
                        interm[x]=inconnu
                psi[t, j] = interm.argmax()  #Récupérer le meilleur état caché     
                delta_precedent[j] = interm[interm.argmax()] * B[lettre_courante, j] 
            #Permutation:
            z=delta_courant
            delta_courant=delta_precedent
            delta_precedent=z
    #Choisir le meilleur état caché pour la dernière observation
    etats_caches.append(delta_courant.argmax())
    for i in range(psi.shape[0]-1, 0, -1): #Récupérer les meilleurs états des observations précédentes
        etats_caches.append(psi[i, etats_caches[-1]])              
    etats_caches.reverse()#Inverser la liste et la renvoyer
    return etats_caches

Tout est prêt pour calculer la précision de HMM2:

In [51]:
cptBienCorrigees = 0
somme = 0
for mot in ensTest: #Pour chaque mot de l'ensemble de test
    motTape, motCorrect=CoderMot(mot, dictTapees, dictCorrectes, inconnu) #Récupérer le mot en tant que codes
    motCorrige =Viterbi2(motTape, N, L, A, B, C, Pi, dictTrigram, observations, inconnu) #Récupérer la séquence d'états cachés retournés par l'algorithme de Viterbi
    cptBienCorrigees += sum(np.array(motCorrect)==np.array(motCorrige)) #Accumuler le nombre de lettres bien corrigés
    somme += len(motTape) #Accumuler le nombre total de lettres 
print("Précision de l'HMM du deuxième ordre={:.2f}%".format(float(cptBienCorrigees)/somme*100))
print ("Nombre de lettres bien corrigées = " + str(cptBienCorrigees))
print ("Nombre de lettres mal corrigées = " + str(somme-cptBienCorrigees))

Précision de l'HMM du deuxième ordre=94.34%
Nombre de lettres bien corrigées = 6906
Nombre de lettres mal corrigées = 414


Nous remarquons qu'effectivement, le modèle HMM du deuxième ordre (94.34%) est plus efficace que HMM du premier ordre (93.18%). Le nombre de lettres bien corrigées qui était égal à 6822 a augmenté jusqu'à 6906, et le nombre de lettres mal corrigées qui était égal à 498 est diminué jusqu'à 414.

Nous avons gagné un peu dans la précision et nous avons perdu un peu en termes de temps de calcul, mais pour ce problème, le temps de calcul n'est pas important, en augmantant la taille de l'historique, le modèle devient de plus en plus complexe et couteûx mais bien sûr plus efficace. Il reste qu'à faire le compromis entre ces deux critères selon le type de problèmes qu'on veut traiter.

| **HMM1** | | **HMM2** |
------------ | -------------
| **Bien corrigées** | **Précision** | **Bien corrigées** | **Précision**
**BDD_10** | 6822/7320 | 93.20% | 6906/7320 | 94.34%
**BDD_20** | 14499/16691 | 86.87%| 14949/16691 | 89.56%

### Critics and evolution: This model is somehow limited. For instance, it only handles substitution errors. Could you describe a way to extend this model to also handle noisy insertion of characters ?  Same question for deletion.

Nous remarquons que ce modèle marche pour les erreurs de substitution seulement, ceci dit que la longueur de la séquence d'observations et la longueur de la séquence d'états est la même. Le problème d'ajout et de suppression de caractères est une limite pour le modèle HMM de base. Cependant, plusieurs méthodes ont vu le jour afin de modifier HMM pour permettre ce genre de traitements. Parmi les travaux récents nous citons [1].
Les auteurs ....

### Unsupervised training: Propose and discuss some ideas to do unsupervised training for this task. For this question you should provide details on : what kind of data, which parameters will be involved, …

1/ L'idée la plus simple est d'utiliser un dictionnaire de mots (français ou anglais...) implémenté sous forme de dictionnaire (structure de données) où les clés sont les mots du langage et les valeurs sont vides. Pour chaque mot tapé par l'utilisateur, nous testons si le mot existe comme clé dans le dictionnaire. Si c'est le cas, aucune correction ne sera faite. Sinon, on peut calculer la similarité entre le mot tapé et les clés du dictionnaires (par distance de Levenshtein par exemple ou par une méthode de recherche d'information comme TF-IDF). La liste des mots similaires ordonnés par ordre décroissant de similarité sont proposés pour la correction du mot.

L'inconvénient majeur de cette approche est la nécessité d'avoir un dictionnaire le plus riche possible de la langue qu'on traite. De plus, le calcul de similarité avec un dictionnaire aussi grand entraîenra un temps de calcul important. Finalement, la décision peut être délicate en cas où certains mots du dictionnaire ont la même valeur de similarité avec le mot tapé.

2/ Puisque la distance entre les mots peut être mesurée par la méthode de Levenshtein , cela peut nous faire penser à utiliser le clustering (par k-means ou k-medoids...) sur le dictionnaire des mots. Après que les regroupements deviennent stables et leurs centroids/medoids sont calculés, nous calculons la distance du le mot tapé par l'utilisateur avec les centroids/medoids des clusters, on l'attribut au cluster le plus proche. Autrement, les mots du cluster choisi seront proposés pour la correction du mot tapé.

L'inconvénient de cette méthode, mise à part la nécessité d'un dictionnaire et le temps de calcul important, c'est que sa qualité dépend de la qualité du clustering, qui dépend elle aussi de l'initialisation aléatoire des centroids/médoids des clusters et du nombre de clusters choisi. 

### Conclusion

Ce projet était pour nous une occasion pour mettre en pratique l'algorithme de Viterbi avec le modèle HMM. La correction des fautes de frappe est un domaine qui peut s'étendre au domaine médical comme par exemple pour la comparaison des séquence d'ADN. L'algorithme de Viterbi n'est qu'une variante de plusieurs algorithmes qui se basent sur des techniques variées telles que le dictionnaire ...etc, chaque méthode a ses avantages et ses inconvénients et chaque méthode est plus appropriées que les autres pour certains types d'applications.

### Références

[1] Noroozi, Vahid, "Probabilistic insertion, deletion and substitution error correction using Markov inference in next generation sequencing reads" (2016).Graduate Theses and Dissertations. 15097. http://lib.dr.iastate.edu/etd/15097